# Processing workflow example

In [1]:
from rime.process_config import *
from rime.rime_functions import *
from rime.utils import *

import dask
import dask.dataframe as dd
# from dask.diagnostics import ProgressBar
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, ProgressBar
from dask.distributed import Client

# from dask.distributed import Client # uncomment this to open Dask client to see performance
import glob
import numpy as np
import pandas as pd
import pyam
import re
import time
import xarray as xr
import yaml

<IPython.core.display.Javascript object>

In [2]:
# Load a yaml file that will help select multiple climate indicator files (not necessarily needed)
with open("indicator_params.yml", "r") as f:
    params = yaml.full_load(f)

In [3]:
dask.config.set(scheduler="processes")
dask.config.set(num_workers=num_workers)
client = Client()  # uncomment this to open Dask client

To see the Dask dasboard, open http://localhost:8787/status in your browser

### Import scenarios data

Load an IAMC scenarios dataset.  We can use the one that comes with the package.
Decide whether using global mean temperature or CO2 mode.  
Assign SSPs if missing and fix duplicate temperatures.  

In [4]:
os.getcwd()

'/home/byers/rime/rime'

In [5]:
df_scens_in = pyam.IamDataFrame('test_data/emissions_temp_AR6_small.xlsx')
dft = df_scens_in.filter(variable=temp_variable) # temp_variable as defined in the config file, = 'AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile'

# Replace & fill missing SSP scenario allocation
dft = ssp_helper(dft, ssp_meta_col="Ssp_family", default_ssp="SSP2")

pyam - INFO: Running in a notebook, setting up a basic logging at level INFO
pyam.core - INFO: Reading file test_data/emissions_temp_AR6_small.xlsx
/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
pyam.core - INFO: Reading meta indicators
/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)


### Build a dataset to compare SSPs and IMPs

In [6]:
scenarios = ["SSP1-26", "SSP2-45", "SSP3-Baseline", "SSP5-Baseline"]
variable = (
    "AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile"
)
ssps = df_scens_in.filter(scenario=scenarios, model="IMAGE*", variable=variable)
imps = df_scens_in.filter(
    IMP_marker=["CurPol", "ModAct", "SP", "GS", "Neg"], variable=variable
)  # ,'Ren-2.0','Neg-2.0'
ssps_imp = ssps.append(imps)
ssps_imp = ssp_helper(ssps_imp, ssp_meta_col="Ssp_family", default_ssp="SSP2")
ssps_imp

/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)


<class 'pyam.core.IamDataFrame'>
Index:
 * model    : COFFEE 1.1, GCAM 5.3, IMAGE 3.0, IMAGE 3.0.1, ... WITCH 5.0 (6)
 * scenario : CO_Bridge, EN_INDCi2030_3000f, ... SusDev_SDP-PkBudg1000 (9)
Timeseries data coordinates:
   region   : World (1)
   variable : ... (1)
   unit     : K (1)
   year     : 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, ... 2100 (86)
Meta indicators:
   Category (object) C3, C7, C6, C8, C1 (5)
   Category_name (object) C3: limit warming to 2°C (>67%), ... (5)
   Category_subset (object) C3x_NZGHGs, C7, C6, C8, C1a_NZGHGs (5)
   Subset_Ch4 (object) Limit to 2C (>67%) immediate 2020 action, ... (3)
   Category_Vetting_historical (object) C3, C7, C6, C8, C1 (5)
   ...

# Generate some maps

## Example 1: multiple IAM scenarios, 1 climate indicator

In [7]:
# Requires some configuration - need to download the data from https://zenodo.org/records/10212339, and set the file paths in the file process_config.py

In [8]:

ind = "pr"
var = "pr"
short = "r10"  # params["indicators"][ind][var]["short_name"]
ssp = "ssp2"
ftype = "abs"

In [9]:
# published data

# # Prepare data
# impact_data_dir = 'test_data//'
# files = glob.glob(f"{impact_data_dir}/{var}*{short}_*{ftype}.nc4")
# mapdata = xr.open_mfdataset(
#     files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gmt"
# )
# mapdata = xr.concat([mapdata[x] for x in mapdata.data_vars], dim='gmt').to_dataset(name=f'{var}_{short}')
# mapdata = tidy_mapdata(mapdata)
# mapdata = mapdata.sortby("gmt").drop_duplicates("gmt")

In [10]:
# impact_data_dir = '/mnt/c/users/byers/iiasa/ECE.prog - Documents/Research Theme - NEXUS/Hotspots_Explorer_2p0/rcre_testing/rime_testdata'
# files = glob.glob(f"{impact_data_dir}/*{var}*{short}_*{ftype}.nc4")
# files

In [11]:

# Prepare data (using split_files on sharepoint)
impact_data_dir = '/mnt/c/users/byers/iiasa/ECE.prog - Documents/Research Theme - NEXUS/Hotspots_Explorer_2p0/rcre_testing/rime_testdata'
files = glob.glob(f"{impact_data_dir}/*{var}*{short}_*{ftype}.nc4")
mapdata = xr.open_mfdataset(
    files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gmt"
)
mapdata
# mapdata = xr.concat([mapdata[x] for x in mapdata.data_vars], dim='gmt').to_dataset(name=f'{var}_{short}')
mapdata = tidy_mapdata(mapdata)
# mapdata = mapdata.sortby("gmt").drop_duplicates("gmt")

/home/byers/rime/rime/utils.py:121: UserWarning: rename 'threshold' to 'gmt' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  mapdata = mapdata.rename({"threshold": "gmt"})


In [13]:
print("Test multiple IAM scenarios, 1 indicator")

start = time.time()

# Run function
map_out_MS = map_transform_gmt_wrapper(
    ssps_imp,
    mapdata,
    years,
    use_dask=True,
    gmt_name="gmt",
    temp_min=1.2,
    temp_max=3.5,
    drawdown_max=0.15,
    interpolation=interpolation,
)

comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in map_out_MS.data_vars}
filename = f"{output_folder_maps}scenario_maps_multiscenario_{ftype}_test_notebook.nc"
map_out_MS.to_netcdf(filename, encoding=encoding)

print("FINISHED Test multiple scenarios, 1 indicator")
print(f"{time.time()-start}")

Test multiple IAM scenarios, 1 indicator
Single indicator mode (multi-scenarios possible)
COFFEE_1_1_EN_NPi2020_400f_lowBECCS
GCAM_5_3_NGFS2_Current Policies
IMAGE_3_0_EN_INDCi2030_3000f
IMAGE_3_0_1_SSP1-26
IMAGE_3_0_1_SSP2-45
IMAGE_3_0_1_SSP3-Baseline
IMAGE_3_0_1_SSP5-Baseline
REMIND-MAgPIE_2_1-4_2_SusDev_SDP-PkBudg1000
WITCH_5_0_CO_Bridge


/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(index)
/home/byers/venvs/seturime/lib/python3.10/site-packages/pyam/core.py:2692: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  index = pd.unique(ind

PermissionError: [Errno 13] Permission denied: '/home/byers/rime/rime/C:\\Users\\byers\\IIASA\\ECE.prog - Documents\\Research Theme - NEXUS\\Hotspots_Explorer_2p0\\rcre_testing\\testing_3\\output\\maps\\scenario_maps_multiscenario_abs_test_notebook.nc'

In [ ]:
map_out_MS

## Example 2: 1 scenario, multiple indicators


### Build combined dataset with multiple indicators

In [19]:
impact_data_dir = '/mnt/c/users/byers/iiasa/ECE.prog - Documents/Research Theme - NEXUS/Hotspots_Explorer_2p0/rcre_testing/rime_testdata'
files = glob.glob(f"{impact_data_dir}/*{var}*{short}_*{ftype}.nc4")
# mapdata = xr.open_mfdataset(
#     files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gmt"
# )
# mapdata = tidy_mapdata(mapdata)
# mapdata
f"{impact_data_dir}/*{var}*{short}_*{ftype}.nc4"

'/mnt/c/users/byers/iiasa/ECE.prog - Documents/Research Theme - NEXUS/Hotspots_Explorer_2p0/rcre_testing/rime_testdata/*cdd*cdd_*abs.nc4'

In [16]:
# Doesn't work -0
# AttributeError: 'DataArray' object has no attribute 'rename_vars'
# lines 230-233 in map_transform_gmt

gmt_name = "gmt"
print("Test 1 scenario, multiple indicators")
start = time.time()
ssp = "ssp2"
mapdata = xr.Dataset()

impact_data_dir = '/mnt/p/watxene/ISIMIP_postprocessed/cse/split_files_fixed'
impact_data_dir = 'test_data'
indicators = ['dri','iavar']
for ind in indicators:

    print(ind)
    files = glob.glob(f"{impact_data_dir}/ISI*{ind}*_{ftype}*.nc4")
        
    a = xr.open_mfdataset(
            files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gwl")
    for x in a.data_vars:
        mapdata[x] = a[x]

mapdata = mapdata.sortby('gwl')
mapdata = mapdata.rename({'threshold':'gwl'})
mapdata = tidy_mapdata(mapdata)
mapdata

Test 1 scenario, multiple indicators


OSError: no files to open

In [72]:
impact_data_dir = '/mnt/p/watxene/ISIMIP_postprocessed/cse/split_files_fixed'

impact_data_dir = 'test_data'
mapdata = xr.Dataset()
indicators = ['dri','iavar']
for ind in indicators:

    print(ind)
    files = glob.glob(f"{impact_data_dir}/ISI*{ind}*_{ftype}*.nc4")
        
    a = xr.open_mfdataset(
            files, preprocess=remove_ssp_from_ds, combine="nested", concat_dim="gwl")
    for x in a.data_vars:
        mapdata[x] = a[x]

mapdata = mapdata.sortby('gwl')
mapdata = mapdata.rename({'threshold':'gwl'})

dri
iavar


/tmp/ipykernel_7165/1205930015.py:20: UserWarning: rename 'threshold' to 'gwl' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  mapdata = mapdata.rename({'threshold':'gwl'})


In [41]:
os.getcwd()

'/home/byers/rime/rime'

### Run transformation (12 impact indicators) (faster with dask=False)

In [ ]:
map_out_MI = map_transform_gmt_wrapper(
    ssps_imp.filter(IMP_marker="CurPol"),
    mapdata,
    years,
    use_dask=False,
    gmt_name="gmt",
)

comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in map_out_MI.data_vars}
filename = f"{output_folder_maps}scenario_maps_multiindicator_{ftype}_test_notebook.nc"
# map_out_MI.to_netcdf(filename, encoding=encoding)

print("FINISHED 1 scenario, multiple indicators")
print(f"{time.time()-start}")

### Data is now transformed to 5 year interval starting 2015

In [ ]:
map_out_MI

## Plot dashboard (takes some minutes)

In [ ]:
filename = "test_map_notebook.html"
plot_maps_dashboard(
    map_out_MI,
    filename=filename,
    year=2063,
    cmap="magma_r",
    shared_axes=True,
    clim=None,
)
os.startfile(filename)

## Process corresponding table data, aggregated by country

### Import the climate impacts database files
Here is an example for sdii - standard precipitation index, and multiple variables, e.g.  
Exposure of land area  
Exposure of population  
Hazard value  
...  


In [ ]:
filesall = glob.glob(fname_input_climate)
files = filesall  # [:6]
f = files[-0]
# load input climate impacts data file
ds = xr.open_mfdataset(f)
ds = ds.sel(year=years)
ds

In [ ]:
varis = list(ds.data_vars.keys())[:lvaris]
dsi = ds[varis]
print(f"# of variables = {len(varis)}")

### Select scenarios data
Load an IAMC scenarios dataset (done previously at the beginning).  
Decide whether using global mean temperature or CO2 mode.  
Assign SSPs if missing and fix duplicate temperatures. 

In [ ]:
mode = "GMT"
if mode == "GMT":
    dfp = df_scens_in.filter(variable=temp_variable)
elif mode == "bypass":
    dfp = prepare_cumulative(df_scens_in, years=years, use_dask=True)
    ts = dfp.timeseries().apply(co2togmt_simple)
    ts = pyam.IamDataFrame(ts)
    ts.rename(
        {
            "variable": {ts.variable[0]: "RIME|GSAT_tcre"},
            "unit": {ts.unit[0]: "°C"},
        },
        inplace=True,
    )
    # Export data to check error and relationships
    # ts.append(dfp).to_csv('c://users//byers//downloads//tcre_gmt_output.csv')
    dfp = ts
    dfp.meta = df_scens_in.meta.copy()
dfp = dfp.filter(year=years)

In [ ]:
# For testing on C1 scenarios
# few_scenarios = True
# very_few_scenarios = False

# if few_scenarios:
#     dfp = dfp.filter(Category=["C1*"])
#     if very_few_scenarios:
#         dfp = dfp.filter(model="REMIND 2.1*", scenario="*")

dfp = ssps_imp

In [ ]:
# pre-prepare the dataset into correct format
#  Assign SSP to meta and select SSP2 in case SSP not present in name
dfp = ssp_helper(dfp, ssp_meta_col="Ssp_family", default_ssp="SSP2", keep_meta=False)

dft = dfp.timeseries()
dft = dft.join(dfp.meta["Ssp_family"]).reset_index()
# dft = dft.apply(fix_duplicate_temps, years=years, axis=1)

### START PROCESSING 

In [ ]:
start = time.time()
year_res = 10
parallel = True
if parallel:
    """
    For parallel processing, convert dft as a wide IAMC pd.Dataframe
    into a dask.DataFrame.
    """
    ddf = dd.from_pandas(dft, npartitions=1000)

    # dfx = dft.iloc[0].squeeze()  # FOR DEBUIGGING THE FUNCTION
    outd = ddf.apply(
        table_impacts_gmt,
        dsi=dsi,
        ssp_meta_col="Ssp_family",
        axis=1,
        meta=("result", None),
    )

    with ProgressBar():
        # try:
        df_new = outd.compute(num_workers=num_workers)
else:
    df_new = dft.apply(table_impacts_gmt, dsi=dsi, axis=1)

expandeddGMT = pd.concat([df_new[x] for x in df_new.index])
print(f" Done:  {time.time()-start}")

filename = f"RIME_output_{region}_{year_res}yr.csv"

# expandedd.to_csv(filename, encoding="utf-8", index=False)
print(f" Saved: {region} yrs={year_res}\n  {time.time()-start}")
print(f"{len(dsi.data_vars)} variables, {len(dfp.meta)} scenarios")

To see the Dask dasboard, open http://localhost:8787/status in your browser

In [ ]:
expandeddGMT = pyam.IamDataFrame(expandeddGMT)

In [ ]:
expandeddGMT.variable

In [ ]:
# model = "IMAG*"
scenario = [
    "CO_Bridge",
    "EN_INDCi2030_3000f",
    "EN_NPi2020_400f_lowBECCS",
    # 'NGFS2_Current Policies',
    "SSP1-26",
    "SSP2-45",
    # 'SSP3-Baseline',
    # 'SSP5-Baseline',
    "SusDev_SDP-PkBudg1000",
]
variable = "RIME|sdii|Hazard|Risk score|Population weighted"
# variable = 'RIME|wsi|Exposure|Population|%'
variable = "RIME|cdd|Exposure|Population|%"
expandeddGMT.filter(variable=variable, scenario=scenario, region="GBR").plot()

In [ ]:
expandeddGMT.filter(variable=variable, scenario=scenario, region="PAK").plot()

In [ ]:
expandeddGMT.variable

### Make SSP & IMP plots for presentation

In [ ]:
imps.scenario

In [ ]:
color_map = {
    "SSP1-26": "AR6-SSP1-2.6",
    "SSP2-45": "AR6-SSP2-4.5",
    "SSP3-Baseline": "AR6-SSP3-7.0",
    "SSP5-Baseline": "AR6-SSP5-8.5",
    "NGFS2_Current Policies": "AR6-IP-CurPol",
    # 'EN_NPi2020_900f': 'AR6-IMP-Neg2.0',
    "EN_INDCi2030_3000f": "AR6-IP-ModAct",
    # 'SSP2_openres_lc_50': 'AR6-IMP-Ren2.0',
    # AR6-IMP-GS',
    "CO_Bridge": "AR6-IMP-GS",
    "EN_NPi2020_400f_lowBECCS": "AR6-IMP-Neg",
    "SusDev_SDP-PkBudg1000": "AR6-IMP-SP",
}

pyam.run_control().update({"color": {"scenario": color_map}})

ssps_imp.filter(
    variable="AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile",
    scenario="SSP*",
).plot(legend=False, fill_between=True, color="scenario", title="")

In [ ]:
ssps_imp.filter(
    variable="AR6 climate diagnostics|Surface Temperature (GSAT)|MAGICCv7.5.3|50.0th Percentile"
).plot(legend=False, fill_between=True, color="scenario", title="")